In [1]:


import numpy as np
import matplotlib.pyplot as plt
from math import log
from numpy.linalg import norm, inv
from scipy.optimize import minimize
import os

# Crear carpeta de salida para las gráficas
os.makedirs("graficas", exist_ok=True)

# ============================================================
# Definición de la función y sus derivados
# ============================================================

def f_val(xy):
    x, y = xy
    val = ((x**2 + y**2)/20)**(np.log(y**2 + 1)/10)
    return val

def grad(xy):
    x, y = xy
    # Derivadas parciales calculadas analíticamente
    base = (x**2 + y**2)/20
    log_term = np.log(y**2 + 1)/10
    if base <= 0:
        return np.array([0.0, 0.0])
    df_dx = (2*x/20) * log_term * base**(log_term - 1)
    df_dy = base**log_term * ( (2*y/(10*(y**2+1))) * np.log(base) + (2*y/20) * log_term / base )
    return np.array([df_dx, df_dy])

def hessian(xy, eps=1e-5):
    n = len(xy)
    H = np.zeros((n,n))
    fx = f_val(xy)
    for i in range(n):
        for j in range(n):
            x_ij1 = xy.copy()
            x_ij2 = xy.copy()
            x_ij3 = xy.copy()
            x_ij4 = xy.copy()
            x_ij1[i] += eps; x_ij1[j] += eps
            x_ij2[i] += eps; x_ij2[j] -= eps
            x_ij3[i] -= eps; x_ij3[j] += eps
            x_ij4[i] -= eps; x_ij4[j] -= eps
            H[i,j] = (f_val(x_ij1) - f_val(x_ij2) - f_val(x_ij3) + f_val(x_ij4)) / (4*eps**2)
    return H

# ============================================================
# Algoritmos de optimización
# ============================================================

def steepest_descent(x0, alpha=0.1, tol=1e-6, max_iter=200):
    x = x0.copy()
    path = [x.copy()]
    for _ in range(max_iter):
        g = grad(x)
        if norm(g) < tol:
            break
        x = x - alpha * g
        path.append(x.copy())
    return np.array(path)

def newton_method(x0, tol=1e-6, max_iter=100):
    x = x0.copy()
    path = [x.copy()]
    for _ in range(max_iter):
        g = grad(x)
        H = hessian(x)
        if norm(g) < tol:
            break
        try:
            dx = -inv(H) @ g
        except np.linalg.LinAlgError:
            dx = -g * 0.1
        x = x + dx
        path.append(x.copy())
    return np.array(path)

def quasi_newton_bfgs(x0):
    res = minimize(lambda v: f_val(v), x0, jac=lambda v: grad(v), method='BFGS')
    return res.x, res.fun, res.nit

# ============================================================
# Visualización de resultados
# ============================================================

def plot_function_surface():
    x = np.linspace(-10, 10, 200)
    y = np.linspace(-10, 10, 200)
    X, Y = np.meshgrid(x, y)
    Z = ((X**2 + Y**2)/20)**(np.log(Y**2 + 1)/10)
    plt.figure(figsize=(8,6))
    plt.contour(X, Y, Z, levels=30, cmap='viridis')
    plt.title("Superficie de la función")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.savefig("graficas/superficie_funcion.png", dpi=300)
    plt.close()

def plot_paths(paths, labels):
    x = np.linspace(-10, 10, 200)
    y = np.linspace(-10, 10, 200)
    X, Y = np.meshgrid(x, y)
    Z = ((X**2 + Y**2)/20)**(np.log(Y**2 + 1)/10)
    plt.figure(figsize=(8,6))
    plt.contour(X, Y, Z, levels=30, cmap='viridis')
    for path, label in zip(paths, labels):
        plt.plot(path[:,0], path[:,1], marker='o', label=label)
    plt.legend()
    plt.title("Trayectorias de los algoritmos")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.savefig("graficas/trayectorias_algoritmos.png", dpi=300)
    plt.close()

# ============================================================
# Ejecución de los algoritmos
# ============================================================

x0 = np.array([5.0, 2.0])

# Descenso del Gradiente
path_sd = steepest_descent(x0, alpha=0.1)
plt.figure()
plot_paths([path_sd], ["Descenso del Gradiente"])

# Método de Newton
path_newton = newton_method(x0)
plt.figure()
plot_paths([path_newton], ["Método de Newton"])

# Quasi-Newton (BFGS)
x_bfgs, f_bfgs, it_bfgs = quasi_newton_bfgs(x0)
print(f"Resultado BFGS: x*={x_bfgs}, f(x*)={f_bfgs}, iteraciones={it_bfgs}")

# Graficar superficie general
plot_function_surface()

# Graficar trayectorias combinadas
plot_paths([path_sd, path_newton], ["Descenso del Gradiente", "Newton"])

print("✅ Gráficas guardadas en la carpeta 'graficas/' correctamente.")

NameError: name 'x2' is not defined